In [177]:
import konlpy
import json
import urllib.request
import requests
import re
from googletrans import Translator

In [178]:
print("보그체로 변환하고 싶은 문장을 입력하시오")
Korean_Text_b = '절대 어제를 후회하지 마라. 인생은 오늘의 내 안에 있고 내일은 스스로 만드는 것이다.'
print(Korean_Text_b)

보그체로 변환하고 싶은 문장을 입력하시오
절대 어제를 후회하지 마라. 인생은 오늘의 내 안에 있고 내일은 스스로 만드는 것이다.


In [179]:
Korean_Text = Korean_Text_b.split(sep=' ')
Korean_Text

['절대',
 '어제를',
 '후회하지',
 '마라.',
 '인생은',
 '오늘의',
 '내',
 '안에',
 '있고',
 '내일은',
 '스스로',
 '만드는',
 '것이다.']

In [180]:
Toknize_Okt = konlpy.tag.Okt()
Toknized_Korean_Okt = []


for i in range(len(Korean_Text)):
    Toknized_Korean_Okt.append(Toknize_Okt.pos(Korean_Text[i]))

    for j in range(len(Toknized_Korean_Okt[i])):
        Toknized_Korean_Okt[i][j] = list(Toknized_Korean_Okt[i][j])

In [181]:
Toknized_Korean_Okt

[[['절대', 'Noun']],
 [['어제', 'Noun'], ['를', 'Josa']],
 [['후회', 'Noun'], ['하지', 'Verb']],
 [['마라', 'Adjective'], ['.', 'Punctuation']],
 [['인생', 'Noun'], ['은', 'Josa']],
 [['오늘', 'Noun'], ['의', 'Josa']],
 [['내', 'Noun']],
 [['안', 'Noun'], ['에', 'Josa']],
 [['있고', 'Adjective']],
 [['내일', 'Noun'], ['은', 'Josa']],
 [['스스로', 'Noun']],
 [['만드는', 'Verb']],
 [['것', 'Noun'], ['이다', 'Josa'], ['.', 'Punctuation']]]

In [182]:
def ends_with_jongsung(letter): # 받침 유무 확인하기 받침이 있으면 True, 없으면 False 반환
    m = re.search(r"[가-힣|ㄴ]?", letter)
    if m:
        n = m.group()[-1]
        return (ord(n)-ord("가")) % 28 > 0 # 나머지가 있으면 받침 있음. 반환값 생김.
    else:
        return None

In [183]:
def josa_change(letter, letter2): # 주격 조사 알맞게 change
    josa = ''
    if letter2 == "을" or letter2 == "를":
        josa = "을" if ends_with_jongsung(letter) else "를" # 앞에 받침 있는 경우, 뒤에는 받침이 없는 경우
    elif letter2 == "이" or letter2 == "가":
        josa = "이" if ends_with_jongsung(letter) else "가"
    elif letter2 == "과" or letter2 == "와":
        josa = "과" if ends_with_jongsung(letter) else "와"
    elif letter2 == "은" or letter2 == "는":
        josa = "은" if ends_with_jongsung(letter) else "는"

    return josa

In [206]:
## PaPago로 보그체 만들기
def Papago_Vogulize(Toknized_Korean_Okt):
    valid_grammer_Okt = ['Noun', 'Exlamation']
    stopwords =['있']


    client_id = "DItIw9wyPPPRDPx4Hy46"
    client_secret = "L6ZEleX9Mp"
    url = "https://openapi.naver.com/v1/papago/n2mt"
    url2 = "https://transliterator.herokuapp.com/"
    sess = requests.Session()

    vogulized_sentence = ''


    for word in range(len(Toknized_Korean_Okt)):
        for w_divided in range(len(Toknized_Korean_Okt[word])):
                if (Toknized_Korean_Okt[word][w_divided][1] in valid_grammer_Okt) and (len(Toknized_Korean_Okt[word][w_divided][0]) >= 1):
                    # print("번역 가능한 품사 단어 : " + Toknized_Korean_Okt[word][w_divided][0])
                    Toknized_Text_English = urllib.parse.quote(Toknized_Korean_Okt[word][w_divided][0])
                    data = "source=ko&target=en&text=" + Toknized_Text_English
                    request = urllib.request.Request(url)
                    request.add_header("X-Naver-Client-Id", client_id)
                    request.add_header("X-Naver-Client-Secret", client_secret)
                    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
                    rescode = response.getcode()

                    if rescode==200:
                        response_body = response.read()
                        json_word = json.loads(response_body.decode("utf-8"))
                        result = json_word['message']['result']['translatedText']
                        Toknized_Korean_Okt[word][w_divided][0]

                        payload = {'input': result}
                        res = sess.post(url2, data=payload)
                        konglish = res.json()['output'].replace('?','')
                        Toknized_Korean_Okt[word][w_divided][0] = konglish
                        # print(konglish)
                        result = re.sub(r'\.', '', result)
                        konglish = str(konglish + '(' + result + ')')
                        vogulized_sentence += konglish

                    else:
                        print("Error")

                else:
                    # print("번역 불가능한 품사 단어 : " + Toknized_Korean_Okt[word][w_divided][0])

                    if Toknized_Korean_Okt[word][w_divided][1] == 'Josa' and Toknized_Korean_Okt[word][w_divided][0] in ['은', '는', '이', '가', '과', '와', '을', '를']:
                        Toknized_Korean_Okt[word][w_divided][0] = josa_change(Toknized_Korean_Okt[word][w_divided-1][0][-1], Toknized_Korean_Okt[word][w_divided][0])

                    vogulized_sentence += str(Toknized_Korean_Okt[word][w_divided][0])

        vogulized_sentence += ' '

    print(vogulized_sentence)
    print(Korean_Text_b)

In [207]:

def Google_Vogulize(Toknized_Korean_Okt):
    valid_grammer_Okt = ['Noun', 'Exlamation']


    stopwords =['있']


    translator = Translator(service_urls=['translate.google.co.kr'])
    url2 = "https://transliterator.herokuapp.com/"
    sess = requests.Session()
    vogulized_sentence = ''

    for word in range(len(Toknized_Korean_Okt)):
        for w_divided in range(len(Toknized_Korean_Okt[word])):
                if Toknized_Korean_Okt[word][w_divided][1] in valid_grammer_Okt and Toknized_Korean_Okt[word][w_divided][0] not in stopwords:
                    # print("번역 가능한 품사 단어 : " + Toknized_Korean_Okt[word][w_divided][0])

                    konglish = translator.translate(Toknized_Korean_Okt[word][w_divided][0], src='ko', dest='en').text
                    result2 = konglish
                    payload = {'input': konglish}
                    res = sess.post(url2, data=payload)
                    konglish = res.json()['output'].replace('?','')
                    Toknized_Korean_Okt[word][w_divided][0] = konglish
                    konglish = str(konglish + '(' + result2 + ')')
                    # print(konglish)

                    vogulized_sentence += konglish

                else:
                    # print("번역 불가능한 품사 단어 : " + Toknized_Korean_Okt[word][w_divided][0])

                    if Toknized_Korean_Okt[word][w_divided][1] == 'Josa' and Toknized_Korean_Okt[word][w_divided][0] in ['은', '는', '이', '가', '과', '와', '을', '를']:
                        Toknized_Korean_Okt[word][w_divided][0] = josa_change(Toknized_Korean_Okt[word][w_divided-1][0][-1], Toknized_Korean_Okt[word][w_divided][0])

                    vogulized_sentence += str(Toknized_Korean_Okt[word][w_divided][0])

        vogulized_sentence += ' '


    print(vogulized_sentence)
    print(Korean_Text_b)

In [211]:
if __name__ == "__main__":
    print("보그체로 변환하고 싶은 문장을 입력하시오")
    Korean_Text_b = '절대 어제를 후회하지 마라. 인생은 오늘의 내 안에 있고 내일은 스스로 만드는 것이다.'
    Korean_Text = Korean_Text_b.split(sep=' ')

    Toknize_Okt = konlpy.tag.Okt()
    Toknized_Korean_Okt = []


    for i in range(len(Korean_Text)):
        Toknized_Korean_Okt.append(Toknize_Okt.pos(Korean_Text[i]))

        for j in range(len(Toknized_Korean_Okt[i])):
            Toknized_Korean_Okt[i][j] = list(Toknized_Korean_Okt[i][j])


    word = Toknized_Korean_Okt
    word2 = Toknized_Korean_Okt
    Papago_Vogulize(word)
    print()
    Google_Vogulize(word2)


보그체로 변환하고 싶은 문장을 입력하시오
네버(Never) 예스터데이(Yesterday)를 레그릿(regret)하지 마라. 라이프(Life)는 투데이(today)의 마이(My) 인사이드(inside)에 있고 토모로(Tomorrow)는 오네젤프(oneself) 만드는 씽(Thing)이다. 
절대 어제를 후회하지 마라. 인생은 오늘의 내 안에 있고 내일은 스스로 만드는 것이다.

네버(Never) 예스터데이(Yester Day)를 레그리트(Legrit)하지 마라. 라이프(life)는 투데이(Today)의 마이(my) 인사이드(Inside)에 있고 토모로(Tomoro)는 오네젤브(Oneselv) 만드는 싱크(Sink)이다. 
절대 어제를 후회하지 마라. 인생은 오늘의 내 안에 있고 내일은 스스로 만드는 것이다.


In [210]:
word

[[['네버', 'Noun']],
 [['예스터데이', 'Noun'], ['를', 'Josa']],
 [['레그리트', 'Noun'], ['하지', 'Verb']],
 [['마라', 'Adjective'], ['.', 'Punctuation']],
 [['라이프', 'Noun'], ['는', 'Josa']],
 [['투데이', 'Noun'], ['의', 'Josa']],
 [['마이', 'Noun']],
 [['인사이드', 'Noun'], ['에', 'Josa']],
 [['있고', 'Adjective']],
 [['토모로', 'Noun'], ['는', 'Josa']],
 [['오네젤브', 'Noun']],
 [['만드는', 'Verb']],
 [['싱크', 'Noun'], ['이다', 'Josa'], ['.', 'Punctuation']]]

In [212]:
word2

[[['네버', 'Noun']],
 [['예스터데이', 'Noun'], ['를', 'Josa']],
 [['레그리트', 'Noun'], ['하지', 'Verb']],
 [['마라', 'Adjective'], ['.', 'Punctuation']],
 [['라이프', 'Noun'], ['는', 'Josa']],
 [['투데이', 'Noun'], ['의', 'Josa']],
 [['마이', 'Noun']],
 [['인사이드', 'Noun'], ['에', 'Josa']],
 [['있고', 'Adjective']],
 [['토모로', 'Noun'], ['는', 'Josa']],
 [['오네젤브', 'Noun']],
 [['만드는', 'Verb']],
 [['싱크', 'Noun'], ['이다', 'Josa'], ['.', 'Punctuation']]]